In [1]:
import pandas as pd
import dill

from multiprocessing import shared_memory
from fb_shared_memory import FbSharedMemory
from fb_dataframe import to_flatbuffer
from test_fb_dataframe import test_to_flatbuffer, test_fb_dataframe_head_correctness, test_fb_dataframe_group_by_sum

In [2]:
df = pd.read_csv("climatechange_tweets_all.csv")

In [3]:
%%time
# Serializing the dataframe with dill, then adding it to shared memory
df_bytestring = dill.dumps(df)
shm_a = shared_memory.SharedMemory(create=True, name = "dill_shm", size=len(df_bytestring) + 1000)
shm_a.buf[:len(df_bytestring)] = df_bytestring
print(len(df_bytestring))

63245615
CPU times: user 4.64 s, sys: 76.4 ms, total: 4.71 s
Wall time: 4.74 s


In [4]:
%%time
# Serializing the dataframe with flatbuffers, then adding it to shared memory
shm = FbSharedMemory()
shm.add_dataframe("df1", df)

CPU times: user 533 µs, sys: 502 µs, total: 1.03 ms
Wall time: 700 µs


In [5]:
df.select_dtypes(include=['int64']).items()

<generator object DataFrame.items at 0x133930200>

In [6]:
# to_flatbuffer(df)
# test_to_flatbuffer()
# test_fb_dataframe_head_correctness()
test_fb_dataframe_group_by_sum()

/Users/sanjitkumar/personal_projects/CollegeIllinois/MlDatasystems/MP3/CS598-MP3-Flatbuffers/test_fb_dataframe.py:40: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f"additional_col_{i}"] = additional_int_cols[i]
/Users/sanjitkumar/personal_projects/CollegeIllinois/MlDatasystems/MP3/CS598-MP3-Flatbuffers/test_fb_dataframe.py:40: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df[f"additional_col_{i}"] = additional_int_cols[i]
/Users/sanjitkumar/personal_projects/CollegeIllinois/MlDatasystems/MP3/CS598-MP3-Flatbuffers/test_fb_dat

df_groupby_dill:
           additional_col_0
int_col                  
0                    4550
1                    3580
2                    4164
3                    3368
4                    3911
5                    8202
6                    5094
7                    3786
8                    5532
9                    6854
10                   3347
df_groupby_fb:
      additional_col_0  int_col
0               4550        0
1               3580        1
2               4164        2
3               3368        3
4               3911        4
5               8202        5
6               5094        6
7               3786        7
8               5532        8
9               6854        9
10              3347       10
--------------------
<class 'pandas.core.frame.DataFrame'>
Index: 11 entries, 0 to 10
Data columns (total 1 columns):
 #   Column            Non-Null Count  Dtype
---  ------            --------------  -----
 0   additional_col_0  11 non-null     int64
dtypes: int64

AssertionError: 